# Generator File: GPT2-Medium Finetuned

## 1. Importing libraries

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
# Install required libraries

!pip install tokenizers
!pip install transformers 

!pip install fastai==2.0.15
!pip install fastai2==0.0.30
!pip install fastcore==1.0.16

!pip install -Uqq fastbook

#!pip install torch==1.6.0 torchvision==0.7.0 

  Using cached https://files.pythonhosted.org/packages/98/2e/d4dcc69f67b4557c8543a4c65d3e136b1929b01136b227ceb986e2596825/fastai-2.0.15-py3-none-any.whl
ERROR: fastbook 0.0.16 has requirement fastai>=2.1, but you'll have fastai 2.0.15 which is incompatible.
  Found existing installation: fastai 2.3.1
    Uninstalling fastai-2.3.1:
      Successfully uninstalled fastai-2.3.1
  Using cached https://files.pythonhosted.org/packages/99/c9/bd299caa1f1c002495bc9ffb98d31605e78a131a2ba3ba66a2682a7ab245/fastcore-1.0.16-py3-none-any.whl
ERROR: nbdev 1.1.14 has requirement fastcore>=1.3.19, but you'll have fastcore 1.0.16 which is incompatible.
ERROR: fastrelease 0.1.11 has requirement fastcore>=1.3.13, but you'll have fastcore 1.0.16 which is incompatible.
ERROR: fastbook 0.0.16 has requirement fastai>=2.1, but you'll have fastai 2.0.15 which is incompatible.
  Found existing installation: fastcore 1.3.20
    Uninstalling fastcore-1.3.20:
      Successfully uninstalled fastcore-1.3.20


In [3]:
# Importing required libraries
import pandas as pd

from fastai.text.all import *

import fastbook
from fastbook import *
fastbook.setup_book()

## 2. Importing model

In [4]:
# Import GPT2 tokenizer
from transformers import GPT2TokenizerFast

# Load pre-trained model (weights)
pretrained_weights = 'gpt2-medium'

# Define tokenizer and model
tokenizer = GPT2TokenizerFast.from_pretrained(pretrained_weights, add_prefix_space=True)    # for documentation: https://huggingface.co/transformers/_modules/transformers/tokenization_gpt2.html

In [5]:
# To process this data to train a model, we need to build a Transform that will be applied lazily.

class TransformersTokenizer(Transform):
    def __init__(self, tokenizer): self.tokenizer = tokenizer
    def encodes(self, x): 
        toks = self.tokenizer.tokenize(x)
        return tensor(self.tokenizer.convert_tokens_to_ids(toks))
    def decodes(self, x): return TitledStr(self.tokenizer.decode(x.cpu().numpy()))

In [6]:
# We use callbacks in case we want to alter the behavior of the training loop 
class DropOutput(Callback):
    def after_pred(self): self.learn.pred = self.pred[0]

In [8]:
# Load model
model_path = "/content/gdrive/MyDrive/NLP/model/gpt2-finedtuned.pkl"

gpt2_tuned = load_learner(model_path)

## 3. Generating Text

In [18]:
# Generate Output

TEMP = 0.6     # Temperature is used to control the randomness of predictions by scaling the logits before applying softmax (small (0.2): model is more confident but also more conservative, large( 1.0): more diversity but also more mistakes)
TOP_K = 40
TOP_P = 0.85
NUM_SEQ = 1

def gen_story(my_model, seed, max_len):

  # take input
  prompt_ids = tokenizer.encode(seed)
  inp = tensor(prompt_ids)[None]#.cuda() # un-do .cuda() if no GPU available
 
  # generate output
  sample_outputs = my_model.generate(
                              inp,
                              do_sample = True, 
                              max_length = max_len,     
                              temperature = TEMP,
                              top_k = TOP_K, 
                              top_p = TOP_P, 
                              num_return_sequences = NUM_SEQ,

  )

  # Print Output
  for i, sample_output in enumerate(sample_outputs):
    print("{}: {}...".format(i, tokenizer.decode(sample_output, skip_special_tokens = True)))
    return

In [19]:
# Run if using GPU and if RuntimeError: Input, output and indices must be on the current device 
#device = "cuda:0"
#gpt2 = gpt2.to(device)

In [20]:
# Provided by evaluation/story-generator application
#seed = "Our story begins with an king that lived in his castle with his queen and their two children and they ruled over a large kingdom of happy people."
#max_len = 350

In [21]:
# Generate output
#my_model = gpt2_tuned                   # insert model for text generation

#gen_story(my_model,seed,max_len)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0:  Our story begins with an king that lived in his castle with his queen and their two children and they ruled over a large kingdom of happy people.    'Oh, Mother, Mother, Mother, Mother, Mother, Mother, Mother,                                                                                                                                                                                                                                                                                                             ...


### Generate other fine-tuned models

In [22]:
# Split a GPT2 model in 4 groups for differential learning rates (Code from Finetuning English GPT2 to any language)

def splitter(model):
    "Split a GPT2 `model` in 3 groups for differential learning rates."
    
    # First layers group : decoder blocks from 0 to 3
    modules = []
    for i in range(4): modules.append(model.transformer.h[i])
    groups = [nn.Sequential(*modules)]

    # Second layers group : decoder blocks from 4 to 7
    modules = []
    for i in range(4,8,1): modules.append(model.transformer.h[i])
    groups = L(groups + [nn.Sequential(*modules)])

    # Third layers group : decoder blocks from 8 to 11
    modules = []
    for i in range(8,12,1): modules.append(model.transformer.h[i])
    groups = L(groups + [nn.Sequential(*modules)])
    
    # Fourth layers group : embeddings matrices wte and wpe + LayerNorm at the model output
    groups = L(groups + [nn.Sequential(model.transformer.wte,model.transformer.wpe,model.transformer.ln_f)])
    
    return groups.map(params)

In [25]:
# Load model
model_path = "/content/gdrive/MyDrive/NLP/model/gpt2_3epoch_lr25e-3.pkl"

gpt2_3epoch_lr25e_3 = load_learner(model_path)

my_model = gpt2_3epoch_lr25e_3                  # insert model for text generation

gen_story(my_model,seed,max_len)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0:  Our story begins with an king that lived in his castle with his queen and their two children and they ruled over a large kingdom of happy people.    He did not know that he was by himself.   The mother duck was very angry at this, but she thought that she would be glad to have some eggs.                                                                                                                                                                                                                                                                                            ...
